In [1225]:
# Import all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest,chi2,RFECV


from sklearn.feature_selection import SelectKBest,chi2,RFECV

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import joblib

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
%matplotlib inline

In [1226]:
# Ignore warnings to see clean output
import warnings
warnings.filterwarnings("ignore")

In [1227]:
# Load the data
df_FP=pd.read_csv("/content/train_data_Total_FP.csv")
df_C=pd.read_csv("/content/train_data_Captain.csv")
df_FPT=pd.read_csv("/content/test_data_Total_FP (1).csv")
df_CT=pd.read_csv("/content/test_data_Captain (1).csv")

In [1228]:
# Check the first few rows of the data
df_FP.describe()

,match_id,season,batting_innings,bowling_innings,Starting_11,prev_runs,prev_balls,prev_sixes,prev_fours,prev_wickets,prev_conceded,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck,Total_FP
count,21616.000000,21616.000000,21616.000000,21616.000000,21616.0,14699.000000,14699.000000,14699.000000,14699.000000,11335.000000,11335.000000,6682.000000,20939.000000,20939.000000,11335.000000,6682.000000,21616.000000,21616.000000,21616.000000
mean,11196.635918,2015.633975,1.497641,1.502359,4.0,19.622559,15.126267,0.759848,1.776720,0.916542,25.932510,1.022897,0.510292,39.281389,3.236568,0.275367,0.071521,49.633142,37.617274
std,6452.702186,4.663043,0.500006,0.500006,0.0,21.306417,13.636834,1.343462,2.307336,0.998369,10.675894,0.628788,0.499906,33.193384,1.017250,0.519249,0.257699,28.865971,31.775817
min,1.000000,2008.000000,1.000000,1.000000,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,-4.000000
25%,5611.750000,2012.000000,1.000000,1.000000,4.0,3.000000,5.000000,0.000000,0.000000,0.000000,18.500000,1.000000,0.000000,12.000000,3.000000,0.000000,0.000000,25.000000,12.000000
50%,11195.500000,2015.000000,1.000000,2.000000,4.0,12.000000,11.000000,0.000000,1.000000,1.000000,26.000000,1.000000,1.000000,31.000000,4.000000,0.000000,0.000000,50.000000,30.000000
75%,16783.250000,2020.000000,2.000000,2.000000,4.0,29.000000,22.000000,1.000000,3.000000,1.000000,33.000000,1.000000,1.000000,57.000000,4.000000,0.000000,0.000000,75.000000,55.000000
max,22362.000000,2023.000000,2.000000,2.000000,4.0,175.000000,73.000000,17.000000,19.000000,6.000000,70.000000,5.000000,1.000000,306.000000,4.000000,4.000000,1.000000,99.000000,306.000000


In [1229]:
# Check all the columns of the dataframe
df_FP.columns

Index(['match_id', 'season', 'match_name', 'home_team', 'away_team', 'venue',
       'batting_innings', 'bowling_innings', 'Player_name', 'Starting_11',
       'bowling_team_bowl', 'batting_team_bat', 'prev_runs', 'prev_balls',
       'prev_sixes', 'prev_fours', 'prev_wickets', 'prev_conceded',
       'prev_Overs_Bowled', 'prev_catches', 'prev_Dream Team', 'prev_Total_FP',
       'prev_overs', 'prev_fielding_heroics', 'prev_duck', 'luck', 'Total_FP'],
      dtype='object')

In [1230]:
# Remove rows with nan values
df_FP = df_FP[df_FP["prev_Total_FP"].notna()]
df_C = df_C[df_C["prev_Total_FP"].notna()]
print(df_C['match_id'].count())
print(df_FP['match_id'].count())


20939
20939


In [1231]:
# Drop unimportant columns
df_FP=df_FP.drop(['Starting_11','match_name','bowling_innings','prev_Overs_Bowled'],axis=1)

df_FPT=df_FPT.drop(['Starting_11','match_name','bowling_innings','prev_Overs_Bowled'],axis=1)


df_FP = df_FP.drop(
    ["prev_overs", "prev_conceded", "prev_fours", "prev_sixes", "prev_runs"], axis=1
)
df_FPT = df_FPT.drop(
    ["prev_overs", "prev_conceded", "prev_fours", "prev_sixes", "prev_runs"], axis=1
)

In [1232]:
df_C=df_C.drop(['Starting_11'],axis=1)
df_CT=df_CT.drop(['Starting_11'],axis=1)

In [1233]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Define encoding function
def enc(df_FP):
  df_FP["home_team"]=label_encoder.fit_transform(df_FP["home_team"])
  df_FP["batting_team_bat"]=label_encoder.fit_transform(df_FP["batting_team_bat"])
  df_FP["bowling_team_bowl"]=label_encoder.fit_transform(df_FP["bowling_team_bowl"])
  df_FP["away_team"]=label_encoder.fit_transform(df_FP["away_team"])
  df_FP["venue"]=label_encoder.fit_transform(df_FP["venue"])
  df_FP["Player_name"]=label_encoder.fit_transform(df_FP["Player_name"])

# Fit and transform the list of teams
enc(df_FP)
enc(df_FPT)
df_C["Captain/Vice Captain"]=label_encoder.fit_transform(df_C["Captain/Vice Captain"])
df_FP

,match_id,season,home_team,away_team,venue,batting_innings,Player_name,bowling_team_bowl,batting_team_bat,prev_runs,...,prev_wickets,prev_conceded,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck,Total_FP
1,2,2008,7,1,7,2.0,0,14,1,26.0,...,NaN,NaN,NaN,1.0,32.0,NaN,NaN,0,35,37.0
2,3,2008,1,0,0,1.0,0,14,1,21.0,...,NaN,NaN,1.0,0.0,37.0,NaN,0.0,0,79,18.0
3,4,2008,12,1,28,1.0,0,14,1,0.0,...,NaN,NaN,2.0,0.0,18.0,NaN,0.0,1,33,36.0
4,5,2008,4,1,11,2.0,0,14,1,20.0,...,NaN,NaN,0.0,1.0,36.0,NaN,1.0,0,82,13.0
5,6,2008,10,1,17,2.0,0,14,1,7.0,...,NaN,NaN,0.0,0.0,13.0,NaN,1.0,0,54,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21610,22357,2023,8,12,13,1.0,644,14,14,NaN,...,NaN,NaN,NaN,0.0,12.0,NaN,NaN,0,10,20.0
21611,22358,2023,8,12,13,2.0,644,14,14,NaN,...,NaN,NaN,NaN,0.0,20.0,NaN,NaN,0,3,12.0
21612,22359,2023,7,13,36,2.0,644,14,14,NaN,...,NaN,NaN,NaN,0.0,12.0,NaN,NaN,0,80,12.0
21613,22360,2023,10,3,17,1.0,644,14,14,NaN,...,NaN,NaN,NaN,0.0,12.0,NaN,NaN,0,4,20.0


In [1234]:
# Fill nan values
df_FP["prev_overs"].fillna(-0.1,inplace=True)
df_FP["balls"]=6*df_FP["prev_overs"].astype(int)+(df_FP["prev_overs"]-df_FP["prev_overs"].astype(int))*10
df_FPT["prev_overs"].fillna(-0.1,inplace=True)
df_FPT["balls"]=6*df_FPT["prev_overs"].astype(int)+(df_FPT["prev_overs"]-df_FPT["prev_overs"].astype(int))*10
print(df_FP["balls"].value_counts())

df_FP["prev_conceded"].fillna(-10, inplace=True)
df_FP["economy"] = (df_FP["prev_conceded"] + 1e-5) / (1e-5 + df_FP["balls"]) * 6
df_FPT["prev_conceded"].fillna(-10, inplace=True)
df_FPT["economy"] = (df_FPT["prev_conceded"] + 1e-5) / (1e-5 + df_FPT["balls"]) * 6
print(df_FP["economy"].value_counts())

df_FP["prev_runs"].fillna(1.0,inplace=True)
df_FP["prev_balls"].fillna(-2.0,inplace=True)
df_FPT["prev_runs"].fillna(1.0,inplace=True)
df_FPT["prev_balls"].fillna(-2.0,inplace=True)

balls
-1.0     9604
 24.0    6330
 18.0    2004
 12.0    1413
 6.0     1006
 22.0      80
 23.0      73
 20.0      57
 19.0      56
 21.0      54
 16.0      34
 17.0      34
 14.0      32
 13.0      31
 15.0      29
 7.0       17
 11.0      17
 5.0       12
 9.0       11
 1.0       10
 10.0      10
 2.0        8
 4.0        7
 8.0        6
 3.0        3
 0.0        1
Name: count, dtype: int64

In [1237]:
# Engineer new features
df_FP["strikerate"]=(1e-5+df_FP["prev_runs"])/(df_FP["prev_balls"]+1e-5)*100
df_FPT["strikerate"]=(1e-5+df_FPT["prev_runs"])/(df_FPT["prev_balls"]+1e-5)*100
print(df_FP["strikerate"].isnull().sum())
df_FP["strikerate"].value_counts()

0


strikerate
-50.000750     6240
 100.000000    1446
 0.001000       656
 50.000250      292
 0.000500       291
               ... 
 274.999891       1
 104.347825       1
 163.934416       1
 152.272715       1
 38.095268        1
Name: count, Length: 2065, dtype: int64

In [1238]:
# Checking changes
df_FP.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20939 entries, 1 to 21614
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   match_id               20939 non-null  int64  
 1   season                 20939 non-null  int64  
 2   home_team              20939 non-null  int64  
 3   away_team              20939 non-null  int64  
 4   venue                  20939 non-null  int64  
 5   batting_innings        20939 non-null  float64
 6   Player_name            20939 non-null  int64  
 7   bowling_team_bowl      20939 non-null  int64  
 8   batting_team_bat       20939 non-null  int64  
 9   prev_runs              20939 non-null  float64
 10  prev_balls             20939 non-null  float64
 11  prev_sixes             14699 non-null  float64
 12  prev_fours             14699 non-null  float64
 13  prev_wickets           11335 non-null  float64
 14  prev_conceded          20939 non-null  float64
 15  prev_ca

In [1239]:
# Model performs better by dropping these columns

# # Filling nan values again
# df_FP["prev_fours"].fillna(-1,inplace=True)
# df_FP["prev_sixes"].fillna(-1,inplace=True)
# df_FP["boundaries"]=df_FP["prev_fours"]+df_FP["prev_sixes"]
# df_FPT["prev_fours"].fillna(-1,inplace=True)
# df_FPT["prev_sixes"].fillna(-1,inplace=True)
# df_FPT["boundaries"]=df_FPT["prev_fours"]+df_FPT["prev_sixes"]
# df_FP["boundaries"].value_counts()

boundaries
-2.0     6240
 0.0     4857
 1.0     2670
 2.0     1859
 3.0     1343
 4.0      986
 5.0      760
 6.0      573
 7.0      452
 8.0      334
 9.0      246
 10.0     194
 11.0     107
 12.0      91
 13.0      69
 14.0      56
 15.0      30
 16.0      21
 17.0      20
 19.0      10
 18.0       8
 20.0       5
 23.0       2
 22.0       2
 21.0       2
 30.0       1
 24.0       1
Name: count, dtype: int64

In [1240]:
# Filling nan values again
df_FP["prev_wickets"].fillna(-1,inplace=True)
df_FP["prev_fielding_heroics"].fillna(0,inplace=True)
df_FP["prev_catches"].fillna(0,inplace=True)
df_FPT["prev_wickets"].fillna(-1,inplace=True)
df_FPT["prev_fielding_heroics"].fillna(0,inplace=True)
df_FPT["prev_catches"].fillna(0,inplace=True)
df_FP["Total_FP"].value_counts().sort_index()

Total_FP
-4.0       11
-3.0        3
-2.0      219
-1.0       25
 0.0      185
         ... 
 233.0      1
 240.0      1
 252.0      1
 260.0      1
 306.0      1
Name: count, Length: 207, dtype: int64

In [1242]:
(df_FP.isna().sum()).count()

22

In [1243]:
df_FP

,match_id,season,home_team,away_team,venue,batting_innings,Player_name,bowling_team_bowl,batting_team_bat,prev_balls,...,prev_Dream Team,prev_Total_FP,prev_fielding_heroics,prev_duck,luck,Total_FP,balls,economy,strikerate,boundaries
1,2,2008,7,1,7,2.0,0,14,1,21.0,...,1.0,32.0,0.0,0,35,37.0,-1.0,60.00054,123.809512,2.0
2,3,2008,1,0,0,1.0,0,14,1,17.0,...,0.0,37.0,0.0,0,79,18.0,-1.0,60.00054,123.529398,3.0
3,4,2008,12,1,28,1.0,0,14,1,1.0,...,0.0,18.0,0.0,1,33,36.0,-1.0,60.00054,0.001000,0.0
4,5,2008,4,1,11,2.0,0,14,1,22.0,...,1.0,36.0,1.0,0,82,13.0,-1.0,60.00054,90.909095,0.0
5,6,2008,10,1,17,2.0,0,14,1,12.0,...,0.0,13.0,1.0,0,54,26.0,-1.0,60.00054,58.333368,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21610,22357,2023,8,12,13,1.0,644,14,14,-2.0,...,0.0,12.0,0.0,0,10,20.0,-1.0,60.00054,-50.000750,-2.0
21611,22358,2023,8,12,13,2.0,644,14,14,-2.0,...,0.0,20.0,0.0,0,3,12.0,-1.0,60.00054,-50.000750,-2.0
21612,22359,2023,7,13,36,2.0,644,14,14,-2.0,...,0.0,12.0,0.0,0,80,12.0,-1.0,60.00054,-50.000750,-2.0
21613,22360,2023,10,3,17,1.0,644,14,14,-2.0,...,0.0,12.0,0.0,0,4,20.0,-1.0,60.00054,-50.000750,-2.0


In [1244]:

x_train=df_FP.drop('Total_FP',axis=1)
y_train= pd.DataFrame()
y_train['Captain/Vice Captain']=df_C['Captain/Vice Captain']
y_train['Total_FP']=df_FP['Total_FP']
x_test= df_FPT

In [1245]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(df_FP.drop('Total_FP',axis=1),pd.concat([df_C['Captain/Vice Captain'], df_FP['Total_FP']],axis=1), test_size=0.35, random_state=42)


In [1246]:
x_train2=x_train1.drop(["home_team","batting_team_bat","bowling_team_bowl","away_team","venue","Player_name"],axis=1)


In [1247]:
y_train

,Captain/Vice Captain,Total_FP
1,1,37.0
2,1,18.0
3,1,36.0
4,1,13.0
5,1,26.0
...,...,...
21610,1,20.0
21611,1,12.0
21612,1,12.0
21613,1,20.0


In [1249]:
x_train.columns

Index(['match_id', 'season', 'home_team', 'away_team', 'venue',
       'batting_innings', 'Player_name', 'bowling_team_bowl',
       'batting_team_bat', 'prev_balls', 'prev_wickets', 'prev_catches',
       'prev_Dream Team', 'prev_Total_FP', 'prev_fielding_heroics',
       'prev_duck', 'luck', 'balls', 'economy', 'strikerate', 'boundaries'],
      dtype='object')

In [1250]:

x_train.isnull().sum()

match_id                 0
season                   0
home_team                0
away_team                0
venue                    0
batting_innings          0
Player_name              0
bowling_team_bowl        0
batting_team_bat         0
prev_balls               0
prev_wickets             0
prev_catches             0
prev_Dream Team          0
prev_Total_FP            0
prev_fielding_heroics    0
prev_duck                0
luck                     0
balls                    0
economy                  0
strikerate               0
boundaries               0
dtype: int64

In [1253]:
# Train the FFNN model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np

seed = 61
tf.random.set_seed(seed)
# Define the model
model = Sequential()

# Input layer with 14  neurons
model.add(Dense(100, input_dim=14, activation='relu'))

# Hidden layer 1
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.4))

# Hidden layer 2
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.3))

# Hidden layer 3
model.add(Dense(15, activation='relu'))
model.add(Dropout(0.5))

# Output layer with 1 neuron
model.add(Dense(1, activation='linear'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Compile the model with the specified optimizer and loss function
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Summary of the model
model.summary()

# Normalize the data
scaler_X = MinMaxScaler()

X_scaled = scaler_X.fit_transform(x_train2.drop('match_id',axis=1))
# Train the model
model.fit(X_scaled, y_train1["Total_FP"], epochs=35, batch_size=10, validation_split=0.2)





Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_270 (Dense)           (None, 100)               1500      
                                                                 
 dense_271 (Dense)           (None, 50)                5050      
                                                                 
 dropout_162 (Dropout)       (None, 50)                0         
                                                                 
 dense_272 (Dense)           (None, 30)                1530      
                                                                 
 dropout_163 (Dropout)       (None, 30)                0         
                                                                 
 dense_273 (Dense)           (None, 15)                465       
                                                                 
 dropout_164 (Dropout)       (None, 15)              

In [1254]:
x_test.fillna(x_test['prev_Total_FP'].mean(),inplace=True)

In [1255]:
y_pred_FP= model.predict(scaler_X.transform(x_test.drop(['match_id',"home_team","batting_team_bat","bowling_team_bowl","away_team","venue","Player_name"],axis=1)))
np.unique(y_pred_FP)

23/23 [==============================] - 0s 1ms/step


array([35.858585, 35.90784 , 35.962177, 35.9648  , 35.97164 , 35.978294,
       36.00919 , 36.016582, 36.03412 , 36.04052 , 36.062283, 36.069603,
       36.07971 , 36.08239 , 36.082764, 36.08623 , 36.08741 , 36.090244,
       36.09113 , 36.091473, 36.092278, 36.10912 , 36.114304, 36.116135,
       36.116188, 36.123116, 36.12498 , 36.130985, 36.13175 , 36.133663,
       36.135075, 36.139317, 36.14005 , 36.14067 , 36.14233 , 36.146717,
       36.150085, 36.15941 , 36.162464, 36.163536, 36.165623, 36.16849 ,
       36.175163, 36.1769  , 36.179424, 36.181602, 36.182583, 36.183243,
       36.1854  , 36.18727 , 36.187782, 36.192944, 36.19516 , 36.195248,
       36.19765 , 36.20134 , 36.20336 , 36.204147, 36.205925, 36.207027,
       36.207886, 36.20818 , 36.208694, 36.20966 , 36.214336, 36.21536 ,
       36.21605 , 36.216305, 36.223362, 36.223705, 36.224106, 36.230637,
       36.231377, 36.236584, 36.24215 , 36.24242 , 36.242443, 36.244232,
       36.24555 , 36.25078 , 36.254593, 36.25782 , 

In [1256]:
y_pred_FP


array([[38.726852],
       [38.969795],
       [40.046883],
       [39.04771 ],
       [36.36939 ],
       [38.534115],
       [36.31499 ],
       [38.886276],
       [40.046883],
       [44.047413],
       [38.89786 ],
       [38.909973],
       [36.205925],
       [38.895554],
       [44.050327],
       [36.259747],
       [36.40879 ],
       [39.04771 ],
       [40.046883],
       [36.179424],
       [36.354694],
       [36.192944],
       [36.230637],
       [39.00151 ],
       [36.39944 ],
       [36.08239 ],
       [38.08704 ],
       [37.260605],
       [36.24242 ],
       [36.38495 ],
       [36.1854  ],
       [36.13175 ],
       [40.046883],
       [40.046883],
       [42.51438 ],
       [36.162464],
       [40.046883],
       [36.254593],
       [36.325333],
       [36.183243],
       [38.864944],
       [39.04771 ],
       [40.046883],
       [38.93936 ],
       [39.39597 ],
       [36.40075 ],
       [36.262493],
       [40.046883],
       [39.486893],
       [37.689495],


In [1257]:
y_pred= model.predict(scaler_X.transform(x_test1.drop(['match_id',"home_team","batting_team_bat","bowling_team_bowl","away_team","venue","Player_name"],axis=1)))
rmse = np.sqrt(mean_squared_error(y_test1["Total_FP"], y_pred))
print("RMSE:", rmse)


230/230 [==============================] - 0s 1ms/step
RMSE: 76.91889847327585


In [1258]:
 np.unique(y_pred)


array([-5956.8755  ,    35.722275,    35.747513, ...,    46.15595 ,
          46.872257,    48.054   ], dtype=float32)

In [1259]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from imblearn.over_sampling import SMOTE, RandomOverSampler,ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.svm import OneClassSVM
from imblearn.combine import SMOTETomek, SMOTEENN

In [1260]:
smote = SMOTE(sampling_strategy='auto', random_state=42)

X_resampled, y_resampled = smote.fit_resample(x_train1,y_train1['Captain/Vice Captain'])

In [1261]:
y_resampled.value_counts()

Captain/Vice Captain
1    12582
2    12582
0    12582
Name: count, dtype: int64

In [1262]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=41,
        max_depth=60,
        bootstrap=True,
        max_features='sqrt',
        class_weight='balanced',
        criterion='gini',
        random_state=40),
    "Decision_tree_classifier":DecisionTreeClassifier(criterion='gini',
                                                      max_depth=8,
                                                      max_features=None,
                                                      random_state=60)
}

for clf_name, clf in classifiers.items():
    clf.fit(X_resampled.drop('match_id',axis=1), y_resampled)
    y_pred_C = clf.predict(x_test.drop('match_id',axis=1))
    y_pred = clf.predict(x_test1.drop('match_id',axis=1))

    print(classification_report(y_test1['Captain/Vice Captain'], y_pred))
    print(confusion_matrix(y_test1['Captain/Vice Captain'], y_pred))
    accuracy = accuracy_score(y_test1['Captain/Vice Captain'], y_pred)
    print("Accuracy:", accuracy*100)

              precision    recall  f1-score   support

           0       0.14      0.01      0.02       294
           1       0.92      1.00      0.96      6751
           2       0.00      0.00      0.00       284

    accuracy                           0.92      7329
   macro avg       0.35      0.34      0.33      7329
weighted avg       0.85      0.92      0.88      7329

[[   3  289    2]
 [  18 6726    7]
 [   1  283    0]]
Accuracy: 91.81334424887434
              precision    recall  f1-score   support

           0       0.08      0.09      0.08       294
           1       0.92      0.93      0.93      6751
           2       0.05      0.04      0.04       284

    accuracy                           0.86      7329
   macro avg       0.35      0.35      0.35      7329
weighted avg       0.86      0.86      0.86      7329

[[  25  257   12]
 [ 270 6275  206]
 [  14  259   11]]
Accuracy: 86.10997407559012


In [1263]:
y_pred_C

array([0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [1264]:
np.unique(y_pred_C)

array([0, 1, 2])

In [1265]:
y_pred_C.shape

(709,)

In [1266]:
g=[]

In [1267]:
for i in y_pred_C:
  if(i==0):
    g.append('C')
  elif (i==1):
    g.append('NC')
  else:
    g.append('VC')

g

['C',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'C',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'C',
 'NC',
 'NC',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'C',
 'NC',
 'C',
 'NC',
 'NC',
 'C',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'C',
 'NC',
 'NC',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'VC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'C',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',


In [1270]:
submission = pd.DataFrame()
submission['match_id']= x_test['match_id']
submission['Total_FP']=y_pred_FP
submission['Captain/Vice Captain']= g

submission.to_csv('loserrr-1.csv', index=False)